In [131]:
#!pip install scikit-learn scipy threadpoolctl ipykernel seaborn

In [132]:
#!pip install llama-index llama-hub openai

In [133]:
#!pip install nltk

In [134]:
#!pip install llama_index

In [135]:
#!pip install spacy

In [136]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [137]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [138]:
import os
os.environ["OPENAI_API_KEY"] = "your keyx"
import openai
openai.api_key = "your key"

In [139]:
import tiktoken
from llama_index.core import ServiceContext
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002").encode, 
                                     verbose = 1)
callback_manager = CallbackManager([token_counter])
service_context = ServiceContext.from_defaults(callback_manager = callback_manager)


C:\Users\Boris\AppData\Local\Temp\ipykernel_10968\204387874.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(callback_manager = callback_manager)


In [140]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage

try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/cache/andrew/sleep')
    index = load_index_from_storage(storage_context, show_progress = True)
    print('loading from disk')
#except:
    documents = SimpleDirectoryReader('assets/AndrewHuberman/sleep').load_data()
    index = VectorStoreIndex.from_documents(documents, show_process = True, service_context = service_context)
    index.storage_context.persist(persist_dir='./storage/cache/andrew/sleep/')
    print('persisting to disk')
    print(f'Token count:{token_counter.total_embedding_token_count}')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 95852
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP

In [141]:
openai.log = "debug"

In [142]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.core.response_synthesizers import get_response_synthesizer

In [143]:
retriever = VectorIndexRetriever(
    index = index,
    similarity_top_k = 6
)

In [144]:
sim_process = SimilarityPostprocessor(similarity_cutoff = 0.83)
key_process = KeywordNodePostprocessor(exclude_keywords = ['newsletter'])

In [145]:
resp_syn = get_response_synthesizer(response_mode='no_text')#'compact'

In [146]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[sim_process, key_process],
    response_synthesizer = resp_syn,
    )

In [147]:
from llama_index.core.response.pprint_utils import pprint_response


In [148]:
#token_counter.reset_counts()
response = query_engine.query("What can I do to sleep better?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 8


print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

In [150]:
pprint_response(response, show_source=True)

Final Response:
______________________________________________________________________
Source Node 1/3
Node ID: d5baf279-3599-4f99-bc27-e5be47c87a88
Similarity: 0.8348870938733585
Text: Now, very briefly, I just want to touch on some tools that are
very commonly used by many people out there, and believe it or not,
there is peer-reviewed science on things like eye masks. Do eye masks
improve your ability to stay asleep? And indeed, they do, provided
they are not too tight and provided that the room is cool enough. Why?
Well, ey...
______________________________________________________________________
Source Node 2/3
Node ID: 69e8924a-fc16-4005-8fea-1b1d9167a398
Similarity: 0.8318823534768527
Text: And I already thought I was sleeping pretty well. The way it
works is that you can cool or heat your mattress according to
different times throughout the night. So for instance, you can cool
your mattress if you tend to run warm and that will help you fall and
stay deeply asleep and then towa

In [151]:
from llama_index.core.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Using both the context information and also using your own knowledge, "
    "answer the question: {query_str}\n"
    "If the context isn't helpful, you can also answer the question on your own.\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [152]:
text_qa_template_str = (
    "You are an Andrew huberman assistant that can read Andrew Huberman podcast notes.\n"
    "Always answer the query only using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [153]:
response = index.as_query_engine(
    text_qa_template = text_qa_template 
).query("How does sleep enhance learning memory?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Embedding Token Usage: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.open

In [154]:
print(response)

Sleep enhances learning and memory by playing a crucial role in motor learning, motor skill learning, and the learning of specific details about specific events. Slow wave sleep, which occurs primarily in the early part of the night, is important for these processes. During slow wave sleep, there is a lack of acetylcholine, which is associated with focus, but an increase in serotonin, which is associated with bliss and wellbeing. Studies have shown that motor learning and the learning of detailed information primarily occur during slow wave sleep, making it essential for consolidating new information and skills.


In [155]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='c566f77b-fd74-46e2-83d0-cdcb3de604cd', embedding=None, metadata={'file_path': 'C:\\Users\\Boris\\Desktop\\llamaindex_scripts\\assets\\AndrewHuberman\\sleep\\05_Understanding_and_Using_Dreams_to_Learn_and_to_Forget_Huberman_Lab_Podcast_5.txt', 'file_name': '05_Understanding_and_Using_Dreams_to_Learn_and_to_Forget_Huberman_Lab_Podcast_5.txt', 'file_type': 'text/plain', 'file_size': 75844, 'creation_date': '2023-10-03', 'last_modified_date': '2024-06-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='596a1fd1-d9e4-445e-b608-ee47a0d7076b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'C:\\Users\\Boris\\Desktop\\llamaindex_scripts\\assets\\AndrewHuberman\\slee

In [156]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Sleep enhances learning and memory by playing a
crucial role in motor learning, motor skill learning, and the learning
of specific details about specific events. Slow wave sleep, which
occurs primarily in the early part of the night, is important for
these processes. During slow wave sleep, there is a lack of
acetylcholine, which is associated with focus, but an increase in
serotonin, which is associated with bliss and wellbeing. Studies have
shown that motor learning and the learning of detailed information
primarily occur during slow wave sleep, making it essential for
consolidating new information and skills.
______________________________________________________________________
Source Node 1/2
Node ID: c566f77b-fd74-46e2-83d0-cdcb3de604cd
Similarity: 0.8611302118631738
Text: In addition, slow wave sleep has been shown to be important for
the learning of detailed information. Now this isn't always cognitive
information. We're gonna talk about cognitive information, b